In [ ]:
import os
import PyPDF2
import openai
import faiss
import numpy as np
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

In [ ]:
# Defina a chave da API OpenAI
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Diretório contendo os PDFs
PDF_DIR = "inputs/"

# Função para extrair texto dos PDFs
def extract_text_from_pdfs(pdfs):
    texts = []
    for file in os.listdir(pdfs):
        if file.endswith(".pdf"):
            with open(os.path.join(pdfs, file), "rb") as f:
                reader = PyPDF2.PdfReader(f)
                text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
                texts.append(text)
    return texts


In [ ]:
# Extração de textos
documents = extract_text_from_pdfs(/workspace/FINANCIAL_CHATBOOT/pdfs)

# Geração de embeddings
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_texts(documents, embeddings)

# Salvando índice para reutilização
vector_store.save_local("faiss_index")

# Carregando o índice salvo
vector_store = FAISS.load_local("faiss_index", embeddings)

# Criando um sistema de perguntas e respostas
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(model_name="gpt-4o"),
    retriever=vector_store.as_retriever()
)

In [ ]:
# Interface simples para consulta
while True:
    query = input("Digite sua pergunta (ou 'sair' para encerrar): ")
    if query.lower() == "sair":
        break
    resposta = qa.run(query)
    print("\nResposta: ", resposta)
